In [3]:
import csv
import pandas as pd
import re
import nltk
from nltk.tokenize import sent_tokenize
import statistics

In [4]:
data = pd.read_csv("../../gen/input/data_gall_with_reviews_corrected.csv", sep = ";", index_col = 0)
#data = pd.read_csv("C:/Users/janva/Documents/Git projects/gall_thesis2/gen/input/beer_wine_data_set.csv", sep = ";", index_col = 0)
#data.reset_index(inplace = True, drop = True)

In [5]:
data

,rating,date,review,type,rating_average,count,title,country,size,percentage,...,main_description,usps,secondary_description,taste,complementaries,grape,label,certificate,producer_description,producer
link,,,,,,,,,,,,,,,,,,,,,
https://www.gall.nl/mucho-mas-tinto-rood-75cl-117196.html,5,2023-03-08,Heerlijke wijn geweldige prijs zeker met aanbi...,Easy Rider,4.7,200,Mucho Más Tinto,Spanje,75CL,14%,...,Kort op hout gerijpte blend van garnacha en te...,"['Bestel voor 22:00 vandaag, morgen in huis', ...","Een wijn gemaakt van druiven uit Toro, La Manc...","specerijen,zachten kers",blokjes kaasen worst,Tempranillo,Bevat sulfur dioxide and sulfits,2 keer bekroond,De familie Solis maakt al sinds 1952 met veel ...,Felix Solis
https://www.gall.nl/mucho-mas-tinto-rood-75cl-117196.html,5,2023-03-08,Erg goede wijn. Zowel de rode als de witte.,Easy Rider,4.7,200,Mucho Más Tinto,Spanje,75CL,14%,...,Kort op hout gerijpte blend van garnacha en te...,"['Bestel voor 22:00 vandaag, morgen in huis', ...","Een wijn gemaakt van druiven uit Toro, La Manc...","specerijen,zachten kers",blokjes kaasen worst,Tempranillo,Bevat sulfur dioxide and sulfits,2 keer bekroond,De familie Solis maakt al sinds 1952 met veel ...,Felix Solis
https://www.gall.nl/mucho-mas-tinto-rood-75cl-117196.html,5,2023-03-07,Geweldig,Easy Rider,4.7,200,Mucho Más Tinto,Spanje,75CL,14%,...,Kort op hout gerijpte blend van garnacha en te...,"['Bestel voor 22:00 vandaag, morgen in huis', ...","Een wijn gemaakt van druiven uit Toro, La Manc...","specerijen,zachten kers",blokjes kaasen worst,Tempranillo,Bevat sulfur dioxide and sulfits,2 keer bekroond,De familie Solis maakt al sinds 1952 met veel ...,Felix Solis
https://www.gall.nl/mucho-mas-tinto-rood-75cl-117196.html,5,2023-03-06,Deze wat boerachtige wijn is heerlijk en zacht...,Easy Rider,4.7,200,Mucho Más Tinto,Spanje,75CL,14%,...,Kort op hout gerijpte blend van garnacha en te...,"['Bestel voor 22:00 vandaag, morgen in huis', ...","Een wijn gemaakt van druiven uit Toro, La Manc...","specerijen,zachten kers",blokjes kaasen worst,Tempranillo,Bevat sulfur dioxide and sulfits,2 keer bekroond,De familie Solis maakt al sinds 1952 met veel ...,Felix Solis
https://www.gall.nl/mucho-mas-tinto-rood-75cl-117196.html,5,2023-03-03,Heerlijke borrelwijn zeker een aanrader.,Easy Rider,4.7,200,Mucho Más Tinto,Spanje,75CL,14%,...,Kort op hout gerijpte blend van garnacha en te...,"['Bestel voor 22:00 vandaag, morgen in huis', ...","Een wijn gemaakt van druiven uit Toro, La Manc...","specerijen,zachten kers",blokjes kaasen worst,Tempranillo,Bevat sulfur dioxide and sulfits,2 keer bekroond,De familie Solis maakt al sinds 1952 met veel ...,Felix Solis
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
https://www.gall.nl/marquis-de-la-coronne-cotes-du-rhone-rood-75cl-420573.html,4,2023-03-10,"Als vanouds, herkenbaar en no nonsens wijn.\nG...",Classic Red,4.3,3,Marquis De La Coronne Cotes Du Rhone,Frankrijk,75CL,13%,...,Rhônewijn gemaakt van de druiven grenache en s...,"['Bestel voor 22:00 vandaag, morgen in huis', ...",Marquis de la Coronne is een Côte du Rhône gem...,fruitig en kruidig,Paté en kipgerechten,"Syrah, Grenache",Bevat sulfur dioxide and sulfits,1 keer bekroond,Cellier des Dauphins is een coöperatie van wij...,Cellier des Dauphins
https://www.gall.nl/marquis-de-la-coronne-cotes-du-rhone-rood-75cl-420573.html,5,2023-03-07,Deze nieuwe Rhone wijn bij Gall&Gall. En gelij...,Classic Red,4.3,3,Marquis De La Coronne Cotes Du Rhone,Frankrijk,75CL,13%,...,Rhônewijn gemaakt van de druiven grenache en s...,"['Bestel voor 22:00 vandaag, morgen in huis', ...",Marquis de la Coronne is een Côte du Rhône gem...,fruitig en kruidig,Paté en kipgerechten,"Syrah, Grenache",Bevat sulfur dioxide and sulfits,1 keer bekroond,Cellier des Dauphins is een coöperatie van wij...,Cellier des Dauphins
https://www.gall.nl/laroche-chablis-magnum-wit-150cl-111988.html,4,2020-12-29,Hele lekkere wijn bij de kerstborrel.,Fresh Prince,4.0,1,Laroc

In [9]:
def return_sentence_length(sentence):
    sentence_length = len(nltk.word_tokenize(sentence))
    return sentence_length

data['number_of_words'] = data.main_description.apply(return_sentence_length)

In [10]:
def return_character_length(sentence):
    sentence_length = len(nltk.word_tokenize(sentence))
    number_characters = len(sentence) - sentence.count(" ")
    average_length = number_characters/sentence_length
    return average_length
data['average_char_length'] = data.main_description.apply(return_character_length)

# Descriptive statistics

In [5]:
print(statistics.mean([len(desc) for desc in list(data['main_description'])]))
print(max([len(desc) for desc in list(data['main_description'])]))
print(min([len(desc) for desc in list(data['main_description'])]))
print(statistics.mean([len(sent_tokenize(desc)) for desc in list(data['main_description'])]))

189.56076900412492
723
51
2.7999410724808484


# Cleaning columns 

In [11]:
#Remove percent sign from alcohol column
data.percentage = data.percentage.str.replace('%','').str.replace(',','.').astype(float)

In [57]:
#Cleaning the size column
##Removing 'cl'
without_cl = [str(i).replace('CL', '') for i in data['size']]
cleaned_cl = []
for size in without_cl:
    size = size.replace(',', '.')
    ##If there are multiple bottles, multiply the sizes
    if 'X' in size:
        sizes = size.split("X")
        result = 1
        for s in sizes:
            result *= float(s)
            
        size = result
        
    elif 'x' in size:
        size = float(size.split("x")[0]) * float(size.split("x")[1])
    elif '%' in size:
        size = None
    cleaned_cl.append(size)
data['cleaned_size'] = cleaned_cl

In [58]:
data['cleaned_size'].unique()

array(['75', '300', 150.0, '150', '37.5', 225.0, 450.0, '100', '50', '25',
       300.0, '225', '198', '30', '133', '33', '132', '180', 198.0, 792.0,
       '135', 132.0, '450', 213.0, 720.0, 100.0, None, '360'],
      dtype=object)

# Grape varieties

In [14]:
#Get varieties
varieties = []
for variety in data['grape']:
    for i in str(variety).split(','):
        varieties.append(i.strip())
set(varieties)

{'Agiorgitiko',
 'Aglianico',
 'Airén',
 'Albariño',
 'Alvarinho',
 'Arneis',
 'Auxerrois',
 'Avesso',
 'Baga',
 'Barbera',
 'Blauer zweigelt',
 'Blaufränkisch',
 'Bonarda',
 'Brachetto',
 'Cabernet franc',
 'Cabernet sauvignon',
 'Cannonau',
 'Carignan',
 'Carignano',
 'Carmenère',
 'Carricante',
 'Chardonnay',
 'Chenin blanc',
 'Cinsault',
 'Clairette',
 'Cortese',
 'Corvina',
 'Dolcetto',
 'Dornfelder',
 'Falanghina',
 'Fiano',
 'Frappato',
 'Friulano',
 'Furmint',
 'Gamay',
 'Garganega',
 'Garnacha',
 'Garnacha blanca',
 'Gewürztraminer',
 'Glera',
 'Godello',
 'Grechetto',
 'Greco',
 'Grenache',
 'Grenache blanc',
 'Grenache noir',
 'Grillo',
 'Gros manseng',
 'Grüner veltliner',
 'Lagrein',
 'Lambrusco',
 'Macabeo',
 'Malbec',
 'Malvasia',
 'Marsanne',
 'Mazuelo',
 'Melon de Bourgogne',
 'Merlot',
 'Molinara',
 'Monastrell',
 'Montepulciano',
 "Montepulciano d'Abruzzo",
 'Moscato',
 'Mourvèdre',
 'Muller thurgau',
 'Muscat de Frontignan',
 'Nebbiolo',
 'Nerello mascalese',
 "Nero

In [37]:
for column in varieties:
    data[column] = 0

In [38]:
for index, row in data.iterrows():
    print(row['grape'])
    if ',' in str(row['grape']):
        all_variety = row['grape'].split(',')
        for variety in all_variety:
            if variety in varieties:
                row[variety] = 1

Tempranillo
Tempranillo
Tempranillo
Tempranillo
Tempranillo
Tempranillo
Tempranillo
Tempranillo
Tempranillo
Tempranillo
Tempranillo
Tempranillo
Tempranillo
Tempranillo
Tempranillo
Tempranillo
Tempranillo
Tempranillo
Tempranillo
Tempranillo
Tempranillo
Tempranillo
Tempranillo
Tempranillo
Tempranillo
Tempranillo
Tempranillo
Tempranillo
Tempranillo
Tempranillo
Tempranillo
Tempranillo
Tempranillo
Tempranillo
Tempranillo
Tempranillo
Tempranillo
Tempranillo
Tempranillo
Tempranillo
Tempranillo
Tempranillo
Tempranillo
Tempranillo
Tempranillo
Tempranillo
Tempranillo
Tempranillo
Tempranillo
Tempranillo
Tempranillo
Tempranillo
Tempranillo
Tempranillo
Tempranillo
Tempranillo
Tempranillo
Tempranillo
Tempranillo
Tempranillo
Tempranillo
Tempranillo
Tempranillo
Tempranillo
Tempranillo
Tempranillo
Tempranillo
Tempranillo
Tempranillo
Tempranillo
Tempranillo
Tempranillo
Tempranillo
Tempranillo
Tempranillo
Tempranillo
Tempranillo
Tempranillo
Tempranillo
Tempranillo
Tempranillo
Tempranillo
Tempranillo
Temp

Primitivo, Montepulciano d'Abruzzo, Nero d'avola, Merlot
Primitivo, Montepulciano d'Abruzzo, Nero d'avola, Merlot
Primitivo, Montepulciano d'Abruzzo, Nero d'avola, Merlot
Primitivo, Montepulciano d'Abruzzo, Nero d'avola, Merlot
Primitivo, Montepulciano d'Abruzzo, Nero d'avola, Merlot
Primitivo, Montepulciano d'Abruzzo, Nero d'avola, Merlot
Primitivo, Montepulciano d'Abruzzo, Nero d'avola, Merlot
Primitivo, Montepulciano d'Abruzzo, Nero d'avola, Merlot
Primitivo, Montepulciano d'Abruzzo, Nero d'avola, Merlot
Primitivo, Montepulciano d'Abruzzo, Nero d'avola, Merlot
Primitivo, Montepulciano d'Abruzzo, Nero d'avola, Merlot
Primitivo, Montepulciano d'Abruzzo, Nero d'avola, Merlot
Primitivo, Montepulciano d'Abruzzo, Nero d'avola, Merlot
Primitivo, Montepulciano d'Abruzzo, Nero d'avola, Merlot
Primitivo, Montepulciano d'Abruzzo, Nero d'avola, Merlot
Primitivo, Montepulciano d'Abruzzo, Nero d'avola, Merlot
Primitivo, Montepulciano d'Abruzzo, Nero d'avola, Merlot
Primitivo, Montepulciano d'Abru

Carmenère
Carmenère
Carmenère
Carmenère
Carmenère
Carmenère
Carmenère
Carmenère
Carmenère
Carmenère
Carmenère
Carmenère
Carmenère
Carmenère
Carmenère
Carmenère
Carmenère
Carmenère
Carmenère
Carmenère
Carmenère
Carmenère
Carmenère
Carmenère
Carmenère
Carmenère
Carmenère
Carmenère
Carmenère
Carmenère
Carmenère
Carmenère
Carmenère
Carmenère
Carmenère
Carmenère
Carmenère
Carmenère
Carmenère
Carmenère
Carmenère
Carmenère
Carmenère
Carmenère
Carmenère
Carmenère
Carmenère
Carmenère
Carmenère
Carmenère
Carmenère
Carmenère
Carmenère
Carmenère
Carmenère
Carmenère
Carmenère
Carmenère
Carmenère
Carmenère
Carmenère
Carmenère
Carmenère
Carmenère
Carmenère
Carmenère
Carmenère
Carmenère
Carmenère
Carmenère
Carmenère
Carmenère
Carmenère
Carmenère
Carmenère
Carmenère
Carmenère
Carmenère
Carmenère
Carmenère
Carmenère
Carmenère
Carmenère
Carmenère
Carmenère
Carmenère
Carmenère
Chardonnay
Chardonnay
Chardonnay
Chardonnay
Chardonnay
Chardonnay
Chardonnay
Chardonnay
Chardonnay
Chardonnay
Chardonnay
Chardonna

Corvina, Molinara, Rondinella
Corvina, Molinara, Rondinella
Corvina, Molinara, Rondinella
Corvina, Molinara, Rondinella
Corvina, Molinara, Rondinella
Corvina, Molinara, Rondinella
Corvina, Molinara, Rondinella
Corvina, Molinara, Rondinella
Corvina, Molinara, Rondinella
Corvina, Molinara, Rondinella
Corvina, Molinara, Rondinella
Corvina, Molinara, Rondinella
Corvina, Molinara, Rondinella
Corvina, Molinara, Rondinella
Corvina, Molinara, Rondinella
Corvina, Molinara, Rondinella
Corvina, Molinara, Rondinella
Corvina, Molinara, Rondinella
Corvina, Molinara, Rondinella
Corvina, Molinara, Rondinella
Corvina, Molinara, Rondinella
Corvina, Molinara, Rondinella
Corvina, Molinara, Rondinella
Corvina, Molinara, Rondinella
Corvina, Molinara, Rondinella
Corvina, Molinara, Rondinella
Corvina, Molinara, Rondinella
Corvina, Molinara, Rondinella
Corvina, Molinara, Rondinella
Corvina, Molinara, Rondinella
Corvina, Molinara, Rondinella
Corvina, Molinara, Rondinella
Corvina, Molinara, Rondinella
Corvina, M

Sauvignon blanc
Sauvignon blanc
Sauvignon blanc
Sauvignon blanc
Sauvignon blanc
Sauvignon blanc
Sauvignon blanc
Sauvignon blanc
Sauvignon blanc
Sauvignon blanc
Sauvignon blanc
Sauvignon blanc
Sauvignon blanc
Sauvignon blanc
Sauvignon blanc
Sauvignon blanc
Sauvignon blanc
Sauvignon blanc
Sauvignon blanc
Sauvignon blanc
Sauvignon blanc
Sauvignon blanc
Sauvignon blanc
Sauvignon blanc
Sauvignon blanc
Sauvignon blanc
Sauvignon blanc
Sauvignon blanc
Sauvignon blanc
Sauvignon blanc
Sauvignon blanc
Sauvignon blanc
Sauvignon blanc
Sauvignon blanc
Sauvignon blanc
Sauvignon blanc
Sauvignon blanc
Sauvignon blanc
Sauvignon blanc
Sauvignon blanc
Sauvignon blanc
Sauvignon blanc
Sauvignon blanc
Sauvignon blanc
Sauvignon blanc
Sauvignon blanc
Sauvignon blanc
Sauvignon blanc
Sauvignon blanc
Sauvignon blanc
Sauvignon blanc
Sauvignon blanc
Sauvignon blanc
Sauvignon blanc
Sauvignon blanc
Sauvignon blanc
Sauvignon blanc
Sauvignon blanc
Sauvignon blanc
Sauvignon blanc
Sauvignon blanc
Sauvignon blanc
Sauvigno

...

Grenache, Syrah, Mourvèdre, Carignan
Grenache, Syrah, Mourvèdre, Carignan
Grenache, Syrah, Mourvèdre, Carignan
Grenache, Syrah, Mourvèdre, Carignan
Grenache, Syrah, Mourvèdre, Carignan
Grenache, Syrah, Mourvèdre, Carignan
Grenache, Syrah, Mourvèdre, Carignan
Grenache, Syrah, Mourvèdre, Carignan
Grenache, Syrah, Mourvèdre, Carignan
Grenache, Syrah, Mourvèdre, Carignan
Grenache, Syrah, Mourvèdre, Carignan
Syrah
Syrah
Syrah
Syrah
Syrah
Syrah
Cabernet franc, Malbec
Cabernet franc, Malbec
Cabernet franc, Malbec
Cabernet franc, Malbec
Cortese
Cortese
Cortese
Cortese
Cortese
Cortese
Cortese
Cortese
Cortese
Cortese
Cortese
Cortese
Cortese
Cortese
Cortese
Cortese
Cortese
Cortese
Cortese
Cortese
Cortese
Cortese
Cortese
Cortese
Cortese
Nebbiolo
Syrah
Syrah
Tempranillo
Tempranillo
Tempranillo
Tempranillo
Tempranillo
Tempranillo
Tempranillo
Tempranillo
Tempranillo
Sauvignon blanc
Sauvignon blanc
Sauvignon blanc
Sauvignon blanc
Glera, Pinot noir
Glera, Pinot noir
Glera, Pinot noir
Pinot blanc
Pinot 

Rondinella, Corvina, Molinara
Rondinella, Corvina, Molinara
Rondinella, Corvina, Molinara
Rondinella, Corvina, Molinara
Rondinella, Corvina, Molinara
Rondinella, Corvina, Molinara
Rondinella, Corvina, Molinara
Rondinella, Corvina, Molinara
Rondinella, Corvina, Molinara
Rondinella, Corvina, Molinara
Rondinella, Corvina, Molinara
Rondinella, Corvina, Molinara
Rondinella, Corvina, Molinara
Rondinella, Corvina, Molinara
Rondinella, Corvina, Molinara
Rondinella, Corvina, Molinara
Rondinella, Corvina, Molinara
Rondinella, Corvina, Molinara
Rondinella, Corvina, Molinara
Rondinella, Corvina, Molinara
Rondinella, Corvina, Molinara
Rondinella, Corvina, Molinara
Rondinella, Corvina, Molinara
Rondinella, Corvina, Molinara
Rondinella, Corvina, Molinara
Rondinella, Corvina, Molinara
Chardonnay, Viognier, Fiano
Chardonnay, Viognier, Fiano
Chardonnay, Viognier, Fiano
Chardonnay, Viognier, Fiano
Chardonnay, Viognier, Fiano
Chardonnay, Viognier, Fiano
Chardonnay, Viognier, Fiano
Chardonnay, Viognier, Fi

Tempranillo, Mazuelo
Tempranillo, Mazuelo
Tempranillo, Mazuelo
Tempranillo, Mazuelo
Nebbiolo
Riesling
Riesling
Chardonnay, Gros manseng
Chardonnay, Gros manseng
Chardonnay, Gros manseng
Garnacha
Garnacha
Garnacha
Garnacha
Garnacha
Garnacha
Garnacha
Garnacha
Garnacha
Garnacha
Garnacha
Grenache, Cinsault, Syrah, Mourvèdre
Grenache, Cinsault, Syrah, Mourvèdre
Grenache, Cinsault, Syrah, Mourvèdre
Pinot blanc
nan
nan
nan
Riesling
Chardonnay
Chardonnay
nan
nan
nan
nan
Sémillon, Sauvignon blanc
Sémillon, Sauvignon blanc
Sémillon, Sauvignon blanc
Sémillon, Sauvignon blanc
Sémillon, Sauvignon blanc
Riesling
Cabernet sauvignon, Merlot, Syrah
Cabernet sauvignon, Merlot, Syrah
Lambrusco
Lambrusco
Lambrusco
Lambrusco
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
Chardonnay
Grenache, Mourvèdre, Syrah
Pinot noir
Pinot noir
nan
nan
Merlot, Petit verdot, Cabernet sauvignon
Grenache, Cinsault, Syrah, Merlot
Grenache, Cinsault, Syrah, Merlot
Grenache, Carignan, Syrah, Mourvèdre
Pinot grigio
Pin

Parellada, Garnacha blanca
Parellada, Garnacha blanca
Parellada, Garnacha blanca
Parellada, Garnacha blanca
Parellada, Garnacha blanca
Parellada, Garnacha blanca
Parellada, Garnacha blanca
Parellada, Garnacha blanca
Parellada, Garnacha blanca
Parellada, Garnacha blanca
Parellada, Garnacha blanca
Parellada, Garnacha blanca
Parellada, Garnacha blanca
Parellada, Garnacha blanca
Parellada, Garnacha blanca
Parellada, Garnacha blanca
Parellada, Garnacha blanca
Parellada, Garnacha blanca
Parellada, Garnacha blanca
Parellada, Garnacha blanca
Parellada, Garnacha blanca
Parellada, Garnacha blanca
Parellada, Garnacha blanca
Parellada, Garnacha blanca
Parellada, Garnacha blanca
Parellada, Garnacha blanca
Parellada, Garnacha blanca
Parellada, Garnacha blanca
Parellada, Garnacha blanca
Parellada, Garnacha blanca
Parellada, Garnacha blanca
Parellada, Garnacha blanca
Parellada, Garnacha blanca
Parellada, Garnacha blanca
Parellada, Garnacha blanca
Cabernet sauvignon
Cabernet sauvignon
Cabernet sauvigno

Corvina, Rondinella
Corvina, Rondinella
Corvina, Rondinella
Corvina, Rondinella
Corvina, Rondinella
Corvina, Rondinella
Corvina, Rondinella
Corvina, Rondinella
Corvina, Rondinella
Corvina, Rondinella
Corvina, Rondinella
Corvina, Rondinella
Corvina, Rondinella
Corvina, Rondinella
Corvina, Rondinella
Corvina, Rondinella
Corvina, Rondinella
Corvina, Rondinella
Corvina, Rondinella
Corvina, Rondinella
Corvina, Rondinella
Corvina, Rondinella
Corvina, Rondinella
Corvina, Rondinella
Corvina, Rondinella
Corvina, Rondinella
Corvina, Rondinella
Corvina, Rondinella
Corvina, Rondinella
Corvina, Rondinella
Corvina, Rondinella
Corvina, Rondinella
Corvina, Rondinella
Corvina, Rondinella
Tempranillo, Mazuelo, Garnacha
Chardonnay
Chardonnay
Chardonnay
Syrah
Chardonnay
Merlot
Merlot
Merlot
Pinot blanc
Gamay
Gamay
Sauvignon blanc
Sauvignon blanc
Sauvignon blanc
Sauvignon blanc
Sauvignon blanc
Sauvignon blanc
Sauvignon blanc
Sauvignon blanc
Sauvignon blanc
Sauvignon blanc
Sauvignon blanc
Merlot
Merlot
Merl

In [39]:
for col in data.iloc[:,22:].columns:
    # check if the column name exists in the value of another column
    data.loc[data['grape'].str.contains(col, na = False), col] = 1

In [40]:
data.iloc[:,22:]

,producer,number_of_words,average_char_length,cleaned_size,Tempranillo,nan,Chardonnay,Pinot grigio,Merlot,Malbec,...,Malvasia,Verdelho,Cannonau,Greco,Muscat de Frontignan,Ugni blanc,Carignano,Garnacha blanca,Schioppettino,Refosco
link,,,,,,,,,,,,,,,,,,,,,
https://www.gall.nl/mucho-mas-tinto-rood-75cl-117196.html,Felix Solis,33,4.727273,75,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
https://www.gall.nl/mucho-mas-tinto-rood-75cl-117196.html,Felix Solis,33,4.727273,75,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
https://www.gall.nl/mucho-mas-tinto-rood-75cl-117196.html,Felix Solis,33,4.727273,75,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
https://www.gall.nl/mucho-mas-tinto-rood-75cl-117196.html,Felix Solis,33,4.727273,75,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
https://www.gall.nl/mucho-mas-tinto-rood-75cl-117196.html,Felix Solis,33,4.727273,75,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
https://www.gall.nl/marquis-de-la-coronne-cotes-du-rhone-rood-75cl-420573.html,Cellier des Dauphins,54,4.611111,75,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
https://www.gall.nl/marquis-de-la-coronne-cotes-du-rhone-rood-75cl-420573.html,Cellier des Dauphins,54,4.611111,75,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
https://www.gall.nl/laroche-chablis-magnum-wit-150cl-111988.html,Domaine Laroche,34,4.911765,150,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Awards

In [2]:
awards = pd.read_csv("../../gen/input/gall_awards.csv", sep = ";")
awards

,product_link,award_link,vintage,grade,contributor
0,https://www.gall.nl/mucho-mas-tinto-rood-75cl-...,https://www.gall.nl/on/demandware.store/Sites-...,Oogstjaar 0000,90 punten,Gilbert and Gaillard
1,https://www.gall.nl/mucho-mas-tinto-rood-75cl-...,https://www.gall.nl/on/demandware.store/Sites-...,NaN,7+,De Grote Hamersma
2,https://www.gall.nl/mucho-mas-blanco-wit-75cl-...,https://www.gall.nl/on/demandware.store/Sites-...,Oogstjaar,8,De Grote Hamersma
3,https://www.gall.nl/la-palma-chardonnay-wit-75...,https://www.gall.nl/on/demandware.store/Sites-...,Oogstjaar 2020,8.5,De Grote Hamersma
4,https://www.gall.nl/la-palma-chardonnay-wit-75...,https://www.gall.nl/on/demandware.store/Sites-...,Oogstjaar 2021,8.5,De Grote Hamersma
...,...,...,...,...,...
4965,https://www.gall.nl/ch%C3%A2teau-barde-haut-ro...,https://www.gall.nl/on/demandware.store/Sites-...,Oogstjaar 2016,p_94 punten,Wine Advocate
4966,https://www.gall.nl/ch%C3%A2teau-barde-haut-ro...,https://www.gall.nl/on/demandware.store/Sites-...,NaN,p_95 punten,Vinous
4967,https://www.gall.nl/ch%C3%A2teau-haut-brion-ro...,https://www.gall.nl/on/demandware.store/Sites-...,Oogstjaar 2012,93 punten,Decanter
4968,https://www.gall.nl/ch%C3%A2teau-haut-brion-ro...,https://www.gall.nl/on/demandware.store/Sites-...,NaN,98 punten,Wine Advocate


In [3]:
awards.vintage.fillna(method = "ffill", inplace = True)

In [7]:
awards.vintage = awards.vintage.str.replace('Oogstjaar ', '').replace("Oogstjaar", "")

In [10]:
awards.contributor.value_counts()

De Grote Hamersma                           1044
James Suckling                               688
Wine Advocate                                400
Decanter                                     387
Tim Atkin                                    293
Wine Spectator                               274
Wine Enthusiast                              241
Vinous                                       176
International Wine Challenge                 164
Falstaff Wine Guide                          152
Jancis Robinson                              141
Mundus Vini                                  126
Platter's Wine Guide                         114
Guía Peñín                                   112
Gambero Rosso                                 91
Descorchados                                  69
International Wine & Spirits Competition      62
IWSC                                          61
Robert Parker                                 58
Int. Wine & Spirits Competition               57
Bettane et Desseauve

In [150]:
def replace_grade(row):
    row = str(row)
    row = row.replace("plus", "+")
    row = str(row).replace("_", ".")
    
    #If range, take the mean
    if re.search(r"[0-9]+-[0-9]+", str(row)):
        row = statistics.mean([int(a) for a in "92-94".split('-')])
    #replace + and - signs    
    if re.search(r'^\d[+-]$', str(row)):
        if "-" in row:
            row = int(row.replace('-', ""))-0.25
        else:
            row = int(row.replace('+', ""))+0.25
    
    #Remove any unnecessary characters
    remove_text = ["punten", " ", "sterren", "p_", "/100", "medaille", "Sterren", "Sterwijn", "+", "-", "p."]
    for rem in remove_text:
        row = str(row).replace(rem, "")
    if row.isalpha():
        row = None
        
    #Return the number
    number = re.search(r"\d+(\.\d+)?", str(row))
    if number == None:
        row = None
    else:
        number = float(number.group(0))
        if number <= 5:
            number = None
        elif number <= 10:
            number = number * 10
        elif number <= 20:
            number = number * 5
        row = number
    
    return row

In [151]:
awards['clean_grade'] = awards.grade.apply(replace_grade)

In [152]:
sum(awards.clean_grade.isnull())

986

In [165]:
awards_link_vintage = awards.groupby(['product_link', "vintage"])['clean_grade'].mean()
awards_link_vintage = pd.DataFrame(awards_link_vintage).sort_values(by=['product_link', 'vintage'])

In [170]:
awards_link_vintage = awards_link_vintage.groupby(['product_link']).ffill()
awards_link_vintage = awards_link_vintage.groupby(['product_link']).bfill()
awards_link_vintage

clean_grade
product_link                                       vintage             
https://www.gall.nl/1000-stories-bourbon-barrel... 2020       85.000000
https://www.gall.nl/1000-stories-bourbon-barrel... 2018       85.000000
                                                   2020       85.000000
https://www.gall.nl/adam-gewurztraminer-grand-c... 2013       91.000000
                                                   2015       91.000000
...                                                                 ...
https://www.gall.nl/zuccardi-aluvional-gualtall... 2018       96.666667
https://www.gall.nl/zuccardi-aluvional-los-chac... 2018       95.500000
https://www.gall.nl/zuccardi-concreto-malbec--r... 2020       94.500000
https://www.gall.nl/zuccardi-tito-rood-75cl-414... 2020       97.000000
                                                   2030       97.000000

[3711 rows x 1 columns]

In [210]:
data['vintage'] = pd.DatetimeIndex(data['date']).year-1

In [211]:
awards_link_vintage.vintage = awards_link_vintage.vintage.replace("", None)

In [193]:
awards_link_vintage.reset_index(inplace = True)

In [225]:
data.reset_index(inplace = True)

In [215]:
awards_link_vintage.vintage = awards_link_vintage.vintage.astype(int)
data.vintage = data.vintage.astype(int)

In [227]:
data = pd.DataFrame(data).sort_values(by=['product_link', 'vintage'])

In [229]:
data['vintage']

0        2021
1        2021
2        2021
3        2019
4        2019
         ... 
13571    2020
13572    2020
13573    2020
13574    2019
13575    2020
Name: vintage, Length: 13576, dtype: int32

In [234]:
links = list(data.product_link.unique())
df = pd.DataFrame(columns = data.columns)
for link in links:
    subset = data[data.product_link == link]
    subset_awards = awards_link_vintage[awards_link_vintage.product_link == link]
    merged_subset = pd.merge_asof(subset, subset_awards, on = "vintage")
    df = pd.concat([df, merged_subset])

In [241]:
df = df[df.columns.drop(list(df.filter(regex='_x|_y')))]

In [243]:
df.to_csv('../../gen/output/data_with_awards.csv', sep = ";")

In [153]:
awards.sort_values(by = ["product_link", "vintage"])

,product_link,award_link,vintage,grade,contributor,clean_grade
1770,https://www.gall.nl/1000-stories-bourbon-barre...,https://www.gall.nl/on/demandware.store/Sites-...,2020,8.5,De Grote Hamersma,85.0
4325,https://www.gall.nl/1000-stories-bourbon-barre...,https://www.gall.nl/on/demandware.store/Sites-...,2018,8.5,De Grote Hamersma,85.0
4326,https://www.gall.nl/1000-stories-bourbon-barre...,https://www.gall.nl/on/demandware.store/Sites-...,2020,8.5,De Grote Hamersma,85.0
2855,https://www.gall.nl/adam-gewurztraminer-grand-...,https://www.gall.nl/on/demandware.store/Sites-...,2013,2 sterren,Guide Hachette,NaN
2856,https://www.gall.nl/adam-gewurztraminer-grand-...,https://www.gall.nl/on/demandware.store/Sites-...,2015,91 punten,Wine Advocate,91.0
...,...,...,...,...,...,...
4954,https://www.gall.nl/zuccardi-aluvional-los-cha...,https://www.gall.nl/on/demandware.store/Sites-...,2018,95 punten,Wine Advocate,95.0
4955,https://www.gall.nl/zuccardi-concreto-malbec--...,https://www.gall.nl/on/demandware.store/Sites-...,2020,94 punten,Tim Atkin,94.0
4956,https://www.gall.nl/zuccardi-concreto-malbec--...,https://www.gall.nl/on/demandware.store/Sites-...,2020,95 punten,James Suckling,95.0
4957,https://www.gall.nl/zuccardi-tito-rood-75cl-41...,https://www.gall.nl/on/demandware.store/Sites-...,2020,97 punten,Decanter,97.0


In [139]:
awards.vintage.value_counts()

2018    766
2017    704
2016    660
2015    635
2019    581
2014    389
2020    337
2013    269
2012    174
2021    169
2011    103
2010     45
0000     38
2009     18
2007     17
2008     15
2022     12
2003      9
          8
2006      6
2005      4
2000      2
2002      1
1990      1
2030      1
2001      1
1999      1
1997      1
0         1
1988      1
2004      1
Name: vintage, dtype: int64

In [125]:
awards.clean_grade.unique()

array([ 90.  ,  72.5 ,  80.  ,  85.  ,  70.  ,    nan,  89.  ,  91.  ,
        87.5 ,  82.5 ,  75.  ,  88.  ,  77.5 ,  40.  ,  45.  ,  93.  ,
        92.  ,  30.  ,  84.  ,  86.  ,  87.  ,  95.  ,  94.  ,  20.  ,
        82.  ,  35.  ,  50.  ,  99.  ,  60.  ,  97.  ,  10.  ,  96.  ,
        92.5 ,  83.  ,  98.  ,  90.5 ,  65.  , 100.  ,  69.  ,  83.75,
        76.  ])

In [44]:
DGH_awards = awards[awards.contributor == "De Grote Hamersma"]

In [45]:
DGH_awards['grade_clean'] = DGH_awards.grade.str.replace('+', '.25')

<ipython-input-45-f89d961e0f33>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DGH_awards['grade_clean'] = DGH_awards.grade.str.replace('+', '.25')


In [46]:
cleaned_grades = []
for grade in DGH_awards['grade_clean']:
    if '-' in grade:
        grade = grade.replace('-', '.75')
        grade = re.findall(r"[-+]?(?:\d*\.*\d+)", str(grade).strip())
        if len(grade) >0:
            grade = float(grade[0])-1
        else:
            grade = None
    else:
        grade = re.findall(r"[-+]?(?:\d*\.*\d+)", str(grade).strip())[0]
    cleaned_grades.append(float(grade))

In [38]:
prices = []
for price in data['price']:
    match1 = re.match(r"^Van: \d+\.\d+ voor: (\d+\.\d+) Per fles$",str(price))
    match2 = re.match(r"^(\d+\.\d+) Per fles$",str(price))
    match3 = re.match(r'Van: \d+\.\d+ voor: (\d+\.\d+)',str(price))
    match4 = re.match(r"^(\d+\.\d+) Per artikel$",str(price))
    match5 = re.match(r"^(\d+\.\d+) Per pack$",str(price))
    if match1:
        price = match1.group(1)
    elif match2:
        price = match2.group(1)
    elif match3:
        price = match3.group(1)
    elif match4:
        price = match4.group(1)
    elif match5:
        price = match5.group(1)
    prices.append(float(price))
data['new_price'] = prices

In [59]:
prices_75 = []
for index, row in data.iterrows():
    #If there are multiple bottles, multiply the sizes
    if 'X' in row['size']:
        size = float(row['size'].split("X")[0])
        price = row['new_price'] * size
        price_75 = price / row['cleaned_size'] * 75
    elif row['cleaned_size'] == None:
        price_75 = None
    else:
        price_75 = float(row['new_price']) / float(row['cleaned_size']) * 75
    prices_75.append(price_75)
data['price_75'] = prices_75

In [47]:
DGH_awards['grade_clean'] = cleaned_grades

<ipython-input-47-544445389c1a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DGH_awards['grade_clean'] = cleaned_grades


In [27]:
cleaned_grades

[7.25,
 8.0,
 8.5,
 8.5,
 8.5,
 7.0,
 7.25,
 7.0,
 7.0,
 8.5,
 8.5,
 8.5,
 8.5,
 8.5,
 8.5,
 8.75,
 9.0,
 8.75,
 8.25,
 8.25,
 8.5,
 8.0,
 8.5,
 8.5,
 7.5,
 8.0,
 8.25,
 8.75,
 8.75,
 8.0,
 7.5,
 7.5,
 7.5,
 8.0,
 8.0,
 7.0,
 8.0,
 8.5,
 8.5,
 8.5,
 9.0,
 8.0,
 7.75,
 7.75,
 8.0,
 8.0,
 8.5,
 8.5,
 8.0,
 8.0,
 8.5,
 8.0,
 8.5,
 8.25,
 8.5,
 9.0,
 7.5,
 7.5,
 8.0,
 7.5,
 8.0,
 8.25,
 8.5,
 8.5,
 8.5,
 7.5,
 8.25,
 8.5,
 8.5,
 7.5,
 7.5,
 7.5,
 7.5,
 8.0,
 8.0,
 8.5,
 7.5,
 8.0,
 8.5,
 9.0,
 8.25,
 8.5,
 7.0,
 8.25,
 8.25,
 8.0,
 7.0,
 7.0,
 7.0,
 8.0,
 8.5,
 7.0,
 8.0,
 8.0,
 8.0,
 8.75,
 8.25,
 7.5,
 7.75,
 8.5,
 8.25,
 7.0,
 8.5,
 8.25,
 8.25,
 7.5,
 7.5,
 8.5,
 9.0,
 9.0,
 7.75,
 8.25,
 7.5,
 8.0,
 8.25,
 8.5,
 8.75,
 7.0,
 7.75,
 8.5,
 8.0,
 8.5,
 8.5,
 8.5,
 8.0,
 8.25,
 8.5,
 8.0,
 7.5,
 8.25,
 8.5,
 8.0,
 8.25,
 8.5,
 8.5,
 7.5,
 7.5,
 8.0,
 8.5,
 8.5,
 8.5,
 8.5,
 -0.241,
 8.5,
 7.5,
 8.5,
 8.5,
 8.25,
 7.75,
 8.0,
 8.25,
 8.5,
 8.5,
 8.5,
 8.0,
 8.0,
 8.5,
 8.0,
 8.0,
 8.0,
 8.

In [50]:
link_grade = DGH_awards.groupby('product_link')['grade_clean'].mean()

In [33]:
data.index

Index(['https://www.gall.nl/mucho-mas-tinto-rood-75cl-117196.html',
       'https://www.gall.nl/mucho-mas-tinto-rood-75cl-117196.html',
       'https://www.gall.nl/mucho-mas-tinto-rood-75cl-117196.html',
       'https://www.gall.nl/mucho-mas-tinto-rood-75cl-117196.html',
       'https://www.gall.nl/mucho-mas-tinto-rood-75cl-117196.html',
       'https://www.gall.nl/mucho-mas-tinto-rood-75cl-117196.html',
       'https://www.gall.nl/mucho-mas-tinto-rood-75cl-117196.html',
       'https://www.gall.nl/mucho-mas-tinto-rood-75cl-117196.html',
       'https://www.gall.nl/mucho-mas-tinto-rood-75cl-117196.html',
       'https://www.gall.nl/mucho-mas-tinto-rood-75cl-117196.html',
       ...
       'https://www.gall.nl/piccini-memoro-rosso-magnum-rood-150cl-154032.html',
       'https://www.gall.nl/piccini-memoro-rosso-magnum-rood-150cl-154032.html',
       'https://www.gall.nl/piccini-memoro-rosso-magnum-rood-150cl-154032.html',
       'https://www.gall.nl/piccini-memoro-rosso-magnum-rood-150cl

In [51]:
data = pd.merge(data,link_grade, left_index = True, right_on = "product_link", how = "left")

# Subtlex-nl import

In [8]:
subtlex = pd.read_excel("../../data/SUBTLEX-NL.xlsx")
subtlex

,Word,FREQcount,CDcount,FREQlow,CDlow,FREQlemma,SUBTLEXWF,Zipf,SUBTLEXCD,Lg10CD,dominant.pos,dominant.pos.freq,dominant.pos.lemma,dominant.pos.lemma.freq,all.pos,all.pos.freq,all.pos.lemma.freq
0,ik,1744062,8054,778704,3125,1744527,39883.0334,7.597064,99.8017,3.9061,VNW,1743609,ik,1743944,.VNW.SPEC.N.VZ.,.1743609.448.4.1.,.1743944.448.134.1.
1,je,1600888,8060,1315051,6535,1600923,36608.9449,7.559864,99.8761,3.9064,VNW,1600798,je,1600798,.VNW.SPEC.N.BW.LID.,.1600798.72.15.2.1.,.1600798.72.50.2.1.
2,het,1068396,8066,780771,5578,1913811,24431.9717,7.384235,99.9504,3.9067,VNW,735390,het,735395,.VNW.LID.SPEC.WW.N.,.735390.332929.53.22.2.,.735395.332929.53.845403.31.
3,de,1061177,8070,903872,6512,1063827,24266.8883,7.381291,100.0000,3.9069,LID,1060098,de,1062748,.LID.VNW.SPEC.VZ.,.1060098.806.272.1.,.1062748.806.272.1.
4,dat,965424,8063,715570,6107,965431,22077.2184,7.340221,99.9133,3.9066,VNW,532576,dat,532576,.VNW.VG.SPEC.N.WW.,.532576.432794.51.2.1.,.532576.432794.51.9.1.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150352,'24,2,2,2,2,2,0.0457,1.832624,0.0248,0.4771,TW,2,'24,2,.TW.,.2.,.2.
150353,'09,2,2,2,2,2,0.0457,1.832624,0.0248,0.4771,TW,2,'09,2,.TW.,.2.,.2.
150354,'02,2,2,2,2,2,0.0457,1.832624,0.0248,0.4771,TW,2,'02,2,.TW.,.2.,.2.
150355,&_fire,2,2,2,2,2,0.0457,1.832624,0.0248,0.4771,SPEC,2,&_fire,2,.SPEC.,.2.,.2.


In [9]:
top_3000 = list(subtlex.loc[:3000,'Word'])

In [32]:
all_words = subtlex[['Word', 'dominant.pos.lemma']]

In [56]:
top_3000_lemma = subtlex['dominant.pos.lemma'].unique()[:3000]

In [71]:
#NDC primary desc
i = 0
scores = []
unfamiliarities = []
sentence_lengths = []
for desc in data['main_description']:
    if i == 0:
        last_desc = desc
        i += 1
        print('first')
    if desc == last_desc:
        pass
    else:
        familiarity = 0
        punct = ['.', ',', '!', '?', ':', ';']
        bag_of_words = desc.split()
        for word in bag_of_words:
            for pun in punct:
                word = word.replace(pun, '').lower()
            #print(word)
            lemma = all_words.loc[all_words['Word'] == word, 'dominant.pos.lemma']
            if len(lemma) > 0:
                if lemma.item() in top_3000_lemma:
                    #print(lemma.item())
                    familiarity += 1
        unfamiliarity = 1- familiarity / len(bag_of_words)
        sentence_len = len(bag_of_words) / len(sent_tokenize(desc))
        score = 0.0496 * sentence_len + 0.1579 * unfamiliarity + 3.6365
    scores.append(score)
    unfamiliarities.append(unfamiliarity)
    sentence_lengths.append(sentence_len)
    last_desc = desc
data['NDC_scores'] = scores
data['familiarity'] = unfamiliarities

fest


In [93]:
#DC-score secondary desc
i = 0
scores2 = []
unfamiliarities2 = []
sentence_lengths2 = []
for desc in data['secondary_description']:
    if i == 0:
        last_desc = desc
        print('first')
    i += 1
    if i % 1000 == 0:
        print(i)
    
    if desc == last_desc:
        pass
    else:
        familiarity = 0
        punct = ['.', ',', '!', '?', ':', ';']
        bag_of_words = str(desc).split()
        for word in bag_of_words:
            for pun in punct:
                word = word.replace(pun, '').lower()
            #print(word)
            lemma = all_words.loc[all_words['Word'] == word, 'dominant.pos.lemma']
            if len(lemma) > 0:
                if lemma.item() in top_3000_lemma:
                    #print(lemma.item())
                    familiarity += 1
        unfamiliarity = 1- familiarity / len(bag_of_words)
        sentence_len = len(bag_of_words) / len(sent_tokenize(str(desc)))
        score = 0.0496 * sentence_len + 0.1579 * unfamiliarity + 3.6365
    scores2.append(score)
    unfamiliarities2.append(unfamiliarity)
    sentence_lengths2.append(sentence_len)
    last_desc = desc

first
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000


# intrinsic product cues

In [40]:
intrinsics = []
varieties = list(set(varieties))
for index, row in data.iterrows():
    count = 0
    description = str(row['main_description']).lower()
    for variety in varieties:
        count += description.count(variety.lower())
    count += description.count(str(row['country']).lower())
    intrinsic = count/row['number_of_words']
    intrinsics.append(intrinsic)
data['intrinsic_cues'] = intrinsics

In [41]:
intrinsics

[0.06060606060606061,
 0.06060606060606061,
 0.06060606060606061,
 0.06060606060606061,
 0.06060606060606061,
 0.06060606060606061,
 0.06060606060606061,
 0.06060606060606061,
 0.06060606060606061,
 0.06060606060606061,
 0.06060606060606061,
 0.06060606060606061,
 0.06060606060606061,
 0.06060606060606061,
 0.06060606060606061,
 0.06060606060606061,
 0.06060606060606061,
 0.06060606060606061,
 0.06060606060606061,
 0.06060606060606061,
 0.06060606060606061,
 0.06060606060606061,
 0.06060606060606061,
 0.06060606060606061,
 0.06060606060606061,
 0.06060606060606061,
 0.06060606060606061,
 0.06060606060606061,
 0.06060606060606061,
 0.06060606060606061,
 0.06060606060606061,
 0.06060606060606061,
 0.06060606060606061,
 0.06060606060606061,
 0.06060606060606061,
 0.06060606060606061,
 0.06060606060606061,
 0.06060606060606061,
 0.06060606060606061,
 0.06060606060606061,
 0.06060606060606061,
 0.06060606060606061,
 0.06060606060606061,
 0.06060606060606061,
 0.06060606060606061,
 0.0606060

# Word frequency

In [23]:
subtlex['Word']

0             ik
1             je
2            het
3             de
4            dat
           ...  
150352       '24
150353       '09
150354       '02
150355    &_fire
150356      &_co
Name: Word, Length: 150357, dtype: object

In [40]:
subtlex[subtlex['Word']==word]['Lg10CD'].iloc[0]

3.9066

In [45]:
average_log_wfs = []
words = list(subtlex['Word'])
size_data = len(data)
i = 0
for index, row in data.iterrows():
    print(f"{i}/{size_data}")
    i += 1
    log_wfs = []
    description = row['secondary_description']
    for word in description.split():
        if word in words:
            log_wf = subtlex[subtlex['Word']==word]['Lg10CD'].iloc[0]
        else:
            log_wf = 0
        log_wfs.append(log_wf)
        average_log_wf = statistics.mean(log_wfs)
    average_log_wfs.append(average_log_wf)  

0/13576
1/13576
2/13576
3/13576
4/13576
5/13576
6/13576
7/13576
8/13576
9/13576
10/13576
11/13576
12/13576
13/13576
14/13576
15/13576
16/13576
17/13576
18/13576
19/13576
20/13576
21/13576
22/13576
23/13576
24/13576
25/13576
26/13576
27/13576
28/13576
29/13576
30/13576
31/13576
32/13576
33/13576
34/13576
35/13576
36/13576
37/13576
38/13576
39/13576
40/13576
41/13576
42/13576
43/13576
44/13576
45/13576
46/13576
47/13576
48/13576
49/13576
50/13576
51/13576
52/13576
53/13576
54/13576
55/13576
56/13576
57/13576
58/13576
59/13576
60/13576
61/13576
62/13576
63/13576
64/13576
65/13576
66/13576
67/13576
68/13576
69/13576
70/13576
71/13576
72/13576
73/13576
74/13576
75/13576
76/13576
77/13576
78/13576
79/13576
80/13576
81/13576
82/13576
83/13576
84/13576
85/13576
86/13576
87/13576
88/13576
89/13576
90/13576
91/13576
92/13576
93/13576
94/13576
95/13576
96/13576
97/13576
98/13576
99/13576
100/13576
101/13576
102/13576
103/13576
104/13576
105/13576
106/13576
107/13576
108/13576
109/13576
110/13576


831/13576
832/13576
833/13576
834/13576
835/13576
836/13576
837/13576
838/13576
839/13576
840/13576
841/13576
842/13576
843/13576
844/13576
845/13576
846/13576
847/13576
848/13576
849/13576
850/13576
851/13576
852/13576
853/13576
854/13576
855/13576
856/13576
857/13576
858/13576
859/13576
860/13576
861/13576
862/13576
863/13576
864/13576
865/13576
866/13576
867/13576
868/13576
869/13576
870/13576
871/13576
872/13576
873/13576
874/13576
875/13576
876/13576
877/13576
878/13576
879/13576
880/13576
881/13576
882/13576
883/13576
884/13576
885/13576
886/13576
887/13576
888/13576
889/13576
890/13576
891/13576
892/13576
893/13576
894/13576
895/13576
896/13576
897/13576
898/13576
899/13576
900/13576
901/13576
902/13576
903/13576
904/13576
905/13576
906/13576
907/13576
908/13576
909/13576
910/13576
911/13576
912/13576
913/13576
914/13576
915/13576
916/13576
917/13576
918/13576
919/13576
920/13576
921/13576
922/13576
923/13576
924/13576
925/13576
926/13576
927/13576
928/13576
929/13576
930/13576


1592/13576
1593/13576
1594/13576
1595/13576
1596/13576
1597/13576
1598/13576
1599/13576
1600/13576
1601/13576
1602/13576
1603/13576
1604/13576
1605/13576
1606/13576
1607/13576
1608/13576
1609/13576
1610/13576
1611/13576
1612/13576
1613/13576
1614/13576
1615/13576
1616/13576
1617/13576
1618/13576
1619/13576
1620/13576
1621/13576
1622/13576
1623/13576
1624/13576
1625/13576
1626/13576
1627/13576
1628/13576
1629/13576
1630/13576
1631/13576
1632/13576
1633/13576
1634/13576
1635/13576
1636/13576
1637/13576
1638/13576
1639/13576
1640/13576
1641/13576
1642/13576
1643/13576
1644/13576
1645/13576
1646/13576
1647/13576
1648/13576
1649/13576
1650/13576
1651/13576
1652/13576
1653/13576
1654/13576
1655/13576
1656/13576
1657/13576
1658/13576
1659/13576
1660/13576
1661/13576
1662/13576
1663/13576
1664/13576
1665/13576
1666/13576
1667/13576
1668/13576
1669/13576
1670/13576
1671/13576
1672/13576
1673/13576
1674/13576
1675/13576
1676/13576
1677/13576
1678/13576
1679/13576
1680/13576
1681/13576
1682/13576

2337/13576
2338/13576
2339/13576
2340/13576
2341/13576
2342/13576
2343/13576
2344/13576
2345/13576
2346/13576
2347/13576
2348/13576
2349/13576
2350/13576
2351/13576
2352/13576
2353/13576
2354/13576
2355/13576
2356/13576
2357/13576
2358/13576
2359/13576
2360/13576
2361/13576
2362/13576
2363/13576
2364/13576
2365/13576
2366/13576
2367/13576
2368/13576
2369/13576
2370/13576
2371/13576
2372/13576
2373/13576
2374/13576
2375/13576
2376/13576
2377/13576
2378/13576
2379/13576
2380/13576
2381/13576
2382/13576
2383/13576
2384/13576
2385/13576
2386/13576
2387/13576
2388/13576
2389/13576
2390/13576
2391/13576
2392/13576
2393/13576
2394/13576
2395/13576
2396/13576
2397/13576
2398/13576
2399/13576
2400/13576
2401/13576
2402/13576
2403/13576
2404/13576
2405/13576
2406/13576
2407/13576
2408/13576
2409/13576
2410/13576
2411/13576
2412/13576
2413/13576
2414/13576
2415/13576
2416/13576
2417/13576
2418/13576
2419/13576
2420/13576
2421/13576
2422/13576
2423/13576
2424/13576
2425/13576
2426/13576
2427/13576

3082/13576
3083/13576
3084/13576
3085/13576
3086/13576
3087/13576
3088/13576
3089/13576
3090/13576
3091/13576
3092/13576
3093/13576
3094/13576
3095/13576
3096/13576
3097/13576
3098/13576
3099/13576
3100/13576
3101/13576
3102/13576
3103/13576
3104/13576
3105/13576
3106/13576
3107/13576
3108/13576
3109/13576
3110/13576
3111/13576
3112/13576
3113/13576
3114/13576
3115/13576
3116/13576
3117/13576
3118/13576
3119/13576
3120/13576
3121/13576
3122/13576
3123/13576
3124/13576
3125/13576
3126/13576
3127/13576
3128/13576
3129/13576
3130/13576
3131/13576
3132/13576
3133/13576
3134/13576
3135/13576
3136/13576
3137/13576
3138/13576
3139/13576
3140/13576
3141/13576
3142/13576
3143/13576
3144/13576
3145/13576
3146/13576
3147/13576
3148/13576
3149/13576
3150/13576
3151/13576
3152/13576
3153/13576
3154/13576
3155/13576
3156/13576
3157/13576
3158/13576
3159/13576
3160/13576
3161/13576
3162/13576
3163/13576
3164/13576
3165/13576
3166/13576
3167/13576
3168/13576
3169/13576
3170/13576
3171/13576
3172/13576

3827/13576
3828/13576
3829/13576
3830/13576
3831/13576
3832/13576
3833/13576
3834/13576
3835/13576
3836/13576
3837/13576
3838/13576
3839/13576
3840/13576
3841/13576
3842/13576
3843/13576
3844/13576
3845/13576
3846/13576
3847/13576
3848/13576
3849/13576
3850/13576
3851/13576
3852/13576
3853/13576
3854/13576
3855/13576
3856/13576
3857/13576
3858/13576
3859/13576
3860/13576
3861/13576
3862/13576
3863/13576
3864/13576
3865/13576
3866/13576
3867/13576
3868/13576
3869/13576
3870/13576
3871/13576
3872/13576
3873/13576
3874/13576
3875/13576
3876/13576
3877/13576
3878/13576
3879/13576
3880/13576
3881/13576
3882/13576
3883/13576
3884/13576
3885/13576
3886/13576
3887/13576
3888/13576
3889/13576
3890/13576
3891/13576
3892/13576
3893/13576
3894/13576
3895/13576
3896/13576
3897/13576
3898/13576
3899/13576
3900/13576
3901/13576
3902/13576
3903/13576
3904/13576
3905/13576
3906/13576
3907/13576
3908/13576
3909/13576
3910/13576
3911/13576
3912/13576
3913/13576
3914/13576
3915/13576
3916/13576
3917/13576

AttributeError: 'float' object has no attribute 'split'

In [ ]:
data['log_wf_2nd'] = average_log_wfs

In [50]:
average_sentence_lengths = []
for index, row in data.iterrows():
    sen_length = len(sent_tokenize(str(row['main_description'])))
    average_length = row['number_of_words']/sen_length
    average_sentence_lengths.append(average_length)

In [51]:
data['average_sentence_length'] = average_sentence_lengths

In [52]:
data.to_csv('../../gen/output/log_wf_set_clean.csv', sep = ";")